# 使うライブラリ

In [ ]:
!pip install git+https://github.com/pfnet-research/xfeat.git

  Cloning https://github.com/pfnet-research/xfeat.git to /tmp/pip-req-build-dvns3bfq
  Running command git clone -q https://github.com/pfnet-research/xfeat.git /tmp/pip-req-build-dvns3bfq
     |████████████████████████████████| 308 kB 8.5 MB/s 
     |████████████████████████████████| 210 kB 50.0 MB/s 
     |████████████████████████████████| 81 kB 8.1 MB/s 
     |████████████████████████████████| 78 kB 8.0 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
     |████████████████████████████████| 146 kB 24.0 MB/s 
     |████████████████████████████████| 113 kB 64.9 MB/s 
  Created wheel for xfeat: filename=xfeat-0.1.1-py3-none-any.whl size=39635 sha256=27e30293c9a8f6500b50670ab2c5aa64f0a6446d12156afcac66787998fbeeb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-1jubdfb_/wheels/71/92/2e/4e706daaf3464ec8e3f53b037c5f3f0346ddc1209eda7d5985
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=e240cef95aa27a69b78a4b05d1ce959030c11f72d6eba0bf

#import

In [ ]:
import pandas as pd
import os
import plotly.express as px

from sklearn.model_selection import KFold
from functools import partial
import lightgbm as lgb
import optuna
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

# pandasで作成するプロットの種類をplotlyに変更
pd.options.plotting.backend = 'plotly'

In [ ]:
cd /content/drive/MyDrive/signate/signate_PM

/content/drive/MyDrive/signate/signate_PM


In [ ]:
ls

lgbm1.ipynb  README.md  submit_sample.csv  test.csv  train.csv  v1.csv  v2.csv


# data取り込み

In [ ]:
train   =pd.read_csv('train.csv')
test   =pd.read_csv('test.csv')
sample =pd.read_csv('submit_sample.csv',names=['id','pm25_mid']) 

train["timestamp"] = pd.to_datetime(train[["year", "month", "day"]])
test["timestamp"] = pd.to_datetime(test[["year", "month", "day"]])

# EDA

In [ ]:
train

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,timestamp
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,...,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,2019-01-01
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,...,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,2019-01-01
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,...,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,2019-01-01
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,...,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370,2019-01-01
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,...,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,195937,2021,12,31,United States,Jacksonville,30.33218,-81.65565,12,0.694,...,2.710,6.125,3.757,12,16.774,22.679,26.058,13.252,16.150,2021-12-31
195937,195938,2021,12,31,United States,Las Vegas,36.17497,-115.13722,14,0.528,...,2.974,6.861,8.354,12,10.432,14.741,15.827,7.078,16.895,2021-12-31
195938,195939,2021,12,31,United States,Milwaukee,43.03890,-87.90647,171,1.975,...,1.087,2.578,0.612,26,2.049,3.531,6.686,5.286,86.299,2021-12-31
195939,195940,2021,12,31,Vietnam,Hanoi,21.02450,105.84117,31,2.613,...,3.058,6.005,6.085,51,1.922,7.443,7.716,4.642,36.523,2021-12-31


In [ ]:
train.columns

Index(['id', 'year', 'month', 'day', 'Country', 'City', 'lat', 'lon', 'co_cnt',
       'co_min', 'co_mid', 'co_max', 'co_var', 'o3_cnt', 'o3_min', 'o3_mid',
       'o3_max', 'o3_var', 'so2_cnt', 'so2_min', 'so2_mid', 'so2_max',
       'so2_var', 'no2_cnt', 'no2_min', 'no2_mid', 'no2_max', 'no2_var',
       'temperature_cnt', 'temperature_min', 'temperature_mid',
       'temperature_max', 'temperature_var', 'humidity_cnt', 'humidity_min',
       'humidity_mid', 'humidity_max', 'humidity_var', 'pressure_cnt',
       'pressure_min', 'pressure_mid', 'pressure_max', 'pressure_var',
       'ws_cnt', 'ws_min', 'ws_mid', 'ws_max', 'ws_var', 'dew_cnt', 'dew_min',
       'dew_mid', 'dew_max', 'dew_var', 'pm25_mid', 'timestamp'],
      dtype='object')

In [ ]:
test

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,timestamp
0,195942,2019,1,1,Australia,Adelaide,-34.92866,138.59863,155,0.091,...,0.098,2.411,10.213,24.608,26,3.057,8.180,15.841,39.705,2019-01-01
1,195943,2019,1,1,Australia,Sydney,-33.86785,151.20732,23,1.037,...,0.101,2.982,8.261,15.957,47,11.090,15.361,19.227,15.857,2019-01-01
2,195944,2019,1,1,Belgium,Brussels,50.85045,4.34878,28,0.096,...,0.218,1.200,5.802,7.584,44,1.055,4.411,6.955,8.415,2019-01-01
3,195945,2019,1,1,Bosnia and Herzegovina,Sarajevo,43.84864,18.35644,122,0.479,...,0.504,1.172,3.861,2.696,16,0.482,0.484,0.932,0.049,2019-01-01
4,195946,2019,1,1,Brazil,São José dos Campos,-23.17944,-45.88694,158,6.198,...,0.409,1.412,3.132,1.754,165,6.253,11.440,12.473,3.546,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53504,249446,2021,12,31,Turkey,Adana,36.98615,35.32531,97,4.674,...,0.201,2.878,7.397,12.130,86,0.451,3.112,4.512,3.938,2021-12-31
53505,249447,2021,12,31,Turkey,Bursa,40.19559,29.06013,123,17.621,...,0.473,1.491,2.974,1.468,115,3.894,6.823,6.861,2.145,2021-12-31
53506,249448,2021,12,31,Turkey,Trabzon,41.00500,39.72694,150,0.128,...,1.001,3.605,5.040,3.965,12,5.269,8.772,12.661,13.201,2021-12-31
53507,249449,2021,12,31,United Kingdom,London,51.50853,-0.12574,21,0.057,...,0.077,0.803,3.884,3.268,34,10.906,10.959,12.744,0.822,2021-12-31


In [ ]:
train.dtypes

id                          int64
year                        int64
month                       int64
day                         int64
Country                    object
City                       object
lat                       float64
lon                       float64
co_cnt                      int64
co_min                    float64
co_mid                    float64
co_max                    float64
co_var                    float64
o3_cnt                      int64
o3_min                    float64
o3_mid                    float64
o3_max                    float64
o3_var                    float64
so2_cnt                     int64
so2_min                   float64
so2_mid                   float64
so2_max                   float64
so2_var                   float64
no2_cnt                     int64
no2_min                   float64
no2_mid                   float64
no2_max                   float64
no2_var                   float64
temperature_cnt             int64
temperature_mi

In [ ]:
# for i in train.columns:
#   fig = px.histogram(train[i], x=i)
#   fig.show()
#   print(train[i].value_counts())
#   print()

In [ ]:
SelectCategorical().fit_transform(train)

,Country,City,timestamp
0,Australia,Brisbane,2019-01-01
1,Australia,Darwin,2019-01-01
2,Australia,Melbourne,2019-01-01
3,Australia,Newcastle,2019-01-01
4,Australia,Perth,2019-01-01
...,...,...,...
195936,United States,Jacksonville,2021-12-31
195937,United States,Las Vegas,2021-12-31
195938,United States,Milwaukee,2021-12-31
195939,Vietnam,Hanoi,2021-12-31


In [ ]:
# for i in test.columns:
#   fig = px.histogram(test[i], x=i)
#   fig.show()
#   print(test[i].value_counts())
#   print()

In [ ]:
len(train['City'].unique())

239

In [ ]:
len(test['City'].unique())

63

# 各都市の距離差分の算出
1 都市データのユニークデータを抽出する
2 ユニーク同士の距離を算出する
3 データフレームに追加

In [ ]:
city_train=train[~train.duplicated(subset=['lat','lon','City'],keep='last')][['lat','lon','City']]
city_test=test[~test.duplicated(subset=['lat','lon','City'],keep='last')][['lat','lon','City']]
city_all=pd.concat([city_train, city_test]).reset_index(drop=True)

In [ ]:
city_all

,lat,lon,City
0,39.73915,-104.98470,Denver
1,36.67642,48.49628,Zanjān
2,33.44838,-112.07404,Phoenix
3,31.76904,35.21633,Jerusalem
4,-26.67313,27.92615,Vereeniging
...,...,...,...
297,36.98615,35.32531,Adana
298,40.19559,29.06013,Bursa
299,41.00500,39.72694,Trabzon
300,51.50853,-0.12574,London


In [ ]:
from geopy.distance import geodesic

dp=[]

for i in range(len(city_all)):
  tmp=[]
  for j in range(len(city_all)):
    
    if i==j:
      tmp.append(0)
    elif i > j:
      tmp.append(dp[j][i])
    else:
      tmp.append(geodesic((city_all['lat'][i],city_all['lon'][i]), (city_all['lat'][j],city_all['lon'][j])).km)
  dp.append(tmp)

In [ ]:
DB=pd.DataFrame(dp,index=city_all['City'],columns=city_all['City'])

In [ ]:
tmp=[]
for i in range(len(train)):
  tmp.append(DB[train['City'][i]].tolist())
train_tmp=pd.DataFrame(tmp,columns=city_all['City'])
train=pd.concat([train,train_tmp],axis=1)
train

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,Suwon,Burgos,Douliu,Tainan,Samut Prakan,Adana,Bursa,Trabzon,London,Haiphong
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,...,7667.145852,17341.249325,6651.660141,6604.290890,7255.901372,14161.706848,14738.854003,13848.174086,16521.956468,7314.561458
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,...,5521.513315,14533.605606,4155.329492,4089.220363,4409.701101,11321.516599,11892.631202,11002.348544,13851.927777,4535.921637
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,...,8518.090881,17318.634364,7273.611831,7211.728282,7336.561487,13949.347251,14587.236662,13776.103784,16898.400596,7631.276215
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,...,8188.175709,17629.283953,7091.196825,7038.341570,7487.932455,14311.048832,14921.133185,14057.147293,16943.293803,7642.882821
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,...,7752.565630,14645.283241,6179.794742,6097.721918,5297.165135,11321.686941,11971.582880,11233.149613,14469.639224,5926.053451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,195937,2021,12,31,United States,Jacksonville,30.33218,-81.65565,12,0.694,...,11948.102354,6880.999798,13574.929894,13661.369774,15137.057946,10085.098828,9435.746001,10078.520524,6872.651908,14269.697266
195937,195938,2021,12,31,United States,Las Vegas,36.17497,-115.13722,14,0.528,...,9662.084716,8878.665382,11175.906408,11257.333764,13345.861168,11352.402920,10792.248008,11082.887072,8421.629694,12317.722198
195938,195939,2021,12,31,United States,Milwaukee,43.03890,-87.90647,171,1.975,...,10435.550185,6583.801712,12062.903044,12149.314544,13673.581033,9451.771978,8829.326247,9322.360225,6301.363900,12773.542463
195939,195940,2021,12,31,Vietnam,Hanoi,21.02450,105.84117,31,2.613,...,2724.909364,9955.225468,1542.353862,1499.257176,992.755145,6940.493643,7434.949798,6528.249489,9249.800671,89.370599


In [ ]:
tmp=[]
for i in range(len(test)):
  tmp.append(DB[test['City'][i]].tolist())
test_tmp=pd.DataFrame(tmp,columns=city_all['City'])
test =pd.concat([test,test_tmp],axis=1)
test

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,Suwon,Burgos,Douliu,Tainan,Samut Prakan,Adana,Bursa,Trabzon,London,Haiphong
0,195942,2019,1,1,Australia,Adelaide,-34.92866,138.59863,155,0.091,...,8084.929399,16667.749474,6764.099392,6698.027315,6703.808344,13297.199788,13933.841021,13121.660154,16258.623869,7036.314682
1,195943,2019,1,1,Australia,Sydney,-33.86785,151.20732,23,1.037,...,8267.549637,17639.356151,7151.808908,7097.760248,7504.190260,14302.929429,14918.456219,14061.391795,16988.909926,7678.768924
2,195944,2019,1,1,Belgium,Brussels,50.85045,4.34878,28,0.096,...,8754.865089,1127.812591,9664.001258,9710.298585,9279.692651,2893.939324,2248.492609,2919.716720,321.282381,9061.599420
3,195945,2019,1,1,Bosnia and Herzegovina,Sarajevo,43.84864,18.35644,122,0.479,...,8509.066157,1798.298397,9134.599969,9166.417199,8356.949747,1624.963764,973.952216,1781.599638,1621.371032,8302.721588
4,195946,2019,1,1,Brazil,São José dos Campos,-23.17944,-45.88694,158,6.198,...,18301.892442,8450.296503,18644.781842,18608.551970,16344.572204,10794.093077,10451.871798,11314.494559,9401.799065,17192.907721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53504,249446,2021,12,31,Turkey,Adana,36.98615,35.32531,97,4.674,...,7775.727455,3370.991296,8061.450118,8077.067127,6924.128570,0.000000,651.560211,586.731866,3214.019029,7024.280961
53505,249447,2021,12,31,Turkey,Bursa,40.19559,29.06013,123,17.621,...,8038.777165,2738.530044,8469.354064,8491.910475,7484.056392,651.560211,0.000000,906.742170,2567.506050,7516.217603
53506,249448,2021,12,31,Turkey,Trabzon,41.00500,39.72694,150,0.128,...,7221.358189,3579.865162,7577.952250,7598.299877,6592.697537,586.731866,906.742170,0.000000,3238.961832,6609.479652
53507,249449,2021,12,31,United Kingdom,London,51.50853,-0.12574,21,0.057,...,8907.735705,1054.568071,9872.405210,9921.453279,9562.396886,3214.019029,2567.506050,3238.961832,0.000000,9316.632589


In [ ]:
# fold = KFold(n_splits=30, shuffle=False)
# encoder = TargetEncoder(
#     input_cols=["timestamp"], 
#     target_col="pm25_mid",
#     fold=fold,
#     output_suffix="_re"
#     )

# train = encoder.fit_transform(train)
# test = encoder.fit_transform(test)
# train[["pm25_mid", "timestamp", "timestamp_re"]].head(12)

In [ ]:
# print(train['timestamp_re'].value_counts())
# print()
# len(train['timestamp'].unique())

# one-hot-encoding

In [ ]:
# カラム指定したい場合は、pd.get_dummies(df_dummy, columns=['workclass'])
df = pd.get_dummies(train,columns=['Country','City'])
test = pd.get_dummies(test,columns=['Country','City'])
df

,id,year,month,day,lat,lon,co_cnt,co_min,co_mid,co_max,...,City_Yunfu,City_Zagreb,City_Zanjān,City_Zenica,City_Zhengzhou,City_Zhuzhou,City_Ürümqi,City_İzmit,City_Łódź,City_Ōita
0,1,2019,1,1,-27.46794,153.02809,38,0.749,2.590,2.633,...,0,0,0,0,0,0,0,0,0,0
1,2,2019,1,1,-12.46113,130.84185,47,2.594,3.181,4.828,...,0,0,0,0,0,0,0,0,0,0
2,3,2019,1,1,-37.81400,144.96332,17,1.190,1.197,2.200,...,0,0,0,0,0,0,0,0,0,0
3,4,2019,1,1,-32.92953,151.78010,63,4.586,11.044,14.802,...,0,0,0,0,0,0,0,0,0,0
4,5,2019,1,1,-31.95224,115.86140,47,4.689,8.681,11.100,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195936,195937,2021,12,31,30.33218,-81.65565,12,0.694,0.995,1.301,...,0,0,0,0,0,0,0,0,0,0
195937,195938,2021,12,31,36.17497,-115.13722,14,0.528,1.256,3.226,...,0,0,0,0,0,0,0,0,0,0
195938,195939,2021,12,31,43.03890,-87.90647,171,1.975,6.627,6.639,...,0,0,0,0,0,0,0,0,0,0
195939,195940,2021,12,31,21.02450,105.84117,31,2.613,2.704,8.767,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def feature_engineering(df):
    cols = df.columns.tolist()

    encoder = Pipeline([
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="+",
                               r=2,
                               output_suffix="_plus"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="*",
                               r=2,
                               output_suffix="_mul"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="-",
                               r=2,
                               output_suffix="_minus"),
        ArithmeticCombinations(input_cols=cols,
                               drop_origin=False,
                               operator="+",
                               r=3,
                               output_suffix="_plus"),
    ])
    return encoder.fit_transform(df)

In [ ]:
df_num=SelectNumerical().fit_transform(df)
test_num=SelectNumerical().fit_transform(test)

In [ ]:
# print("After adding interaction features:")
# df_num = feature_engineering(df_num)

In [ ]:
LGBM_PARAMS = {
        "objective": "regression",
        "metric": "RMSE",
        "verbosity": -1,
}


def objective(df, selector, trial):
    selector.set_trial(trial)
    selector.fit(df)
    input_cols = selector.get_selected_cols()

    # Hyper Parameter Tuning するパラメータと範囲を設定する
    lgbm_params = {
        'num_leaves': trial.suggest_int("num_leaves", 3, 10),
        'max_depth': trial.suggest_int("max_depth", 3, 10),
    }
    lgbm_params.update(LGBM_PARAMS)

    # Evaluate with selected columns
    train_set = lgb.Dataset(df[input_cols], label=df["pm25_mid"])
    scores = lgb.cv(lgbm_params, train_set, num_boost_round=200, stratified=False, seed=1)
    
    rmse_score = scores['rmse-mean'][-1]
    return rmse_score


# 特徴量探索のための説明変数を設定する
# encoded_train_df は、No.7で作成したDataFrameと同じ
input_cols = list(df_num.columns)
input_cols.remove("pm25_mid")


# 特徴量探索器を作成する
selector = GBDTFeatureExplorer(
    input_cols=input_cols,
    target_col="pm25_mid",
    fit_once=True,
    threshold_range=(0.8, 1.0),
    lgbm_params=LGBM_PARAMS,
)

# Hyper Parameter Tuning を行う
study = optuna.create_study(direction="minimize")
study.optimize(partial(objective, df_num, selector), n_trials=10)

# 選択された特徴量を確認する
selector.from_trial(study.best_trial)
print("Selected columns:", selector.get_selected_cols())

[I 2022-05-01 22:44:11,728] A new study created in memory with name: no-name-1569fbeb-b58e-4c16-82d0-82772f027b18
[I 2022-05-01 22:46:42,612] Trial 0 finished with value: 22.341255243481747 and parameters: {'GBDTFeatureSelector.threshold': 0.9312304600477044, 'num_leaves': 4, 'max_depth': 8}. Best is trial 0 with value: 22.341255243481747.
[I 2022-05-01 22:48:13,283] Trial 1 finished with value: 23.024338027344637 and parameters: {'GBDTFeatureSelector.threshold': 0.8980306261365995, 'num_leaves': 3, 'max_depth': 6}. Best is trial 0 with value: 22.341255243481747.
[I 2022-05-01 22:50:33,963] Trial 2 finished with value: 21.16031987663184 and parameters: {'GBDTFeatureSelector.threshold': 0.8071105364595893, 'num_leaves': 8, 'max_depth': 9}. Best is trial 2 with value: 21.16031987663184.
[I 2022-05-01 22:52:48,802] Trial 3 finished with value: 21.19688678296941 and parameters: {'GBDTFeatureSelector.threshold': 0.8744665434894967, 'num_leaves': 8, 'max_depth': 5}. Best is trial 2 with valu

Selected columns: ['co_mid', 'month', 'no2_mid', 'no2_min', 'o3_mid', 'co_min', 'so2_mid', 'o3_max', 'so2_min', 'co_var', 'o3_var', 'pressure_min', 'temperature_max', 'ws_min', 'temperature_mid', 'humidity_mid', 'id', 'co_max', 'so2_var', 'pressure_max', 'so2_max', 'Ürümqi', 'humidity_min', 'o3_min', 'so2_cnt', 'no2_var', 'pressure_cnt', 'Lampang', 'temperature_min', 'no2_cnt', 'dew_min', 'ws_mid', 'dew_max', 'Vereeniging', 'o3_cnt', 'pressure_var', 'no2_max', 'Chandigarh', 'Bhopal', 'ws_cnt', 'dew_mid', 'Delhi', 'ws_var', 'dew_cnt', 'Lhasa', 'Gandhinagar', 'Novosibirsk', 'temperature_cnt', 'humidity_cnt', 'ws_max', 'temperature_var', 'co_cnt', 'Bandar Abbas', 'Shillong', 'lat', 'Qingdao', 'Zanjān', 'San Jose', 'Adapazarı', 'humidity_max', 'Samsun', 'Lucknow', 'Mumbai', 'Antakya', 'Worcester', 'Jerusalem', 'Jinan', 'Patna', 'Hyderabad', 'Changsha', 'Calama', 'Thiruvananthapuram', 'Toluca', 'Jackson', 'Nanchang', 'Phoenix', 'pressure_mid', 'Guadalajara', 'Visakhapatnam', 'Oaxaca', 'Denv

In [ ]:
print(study.best_params)
print(study.best_value)

{'GBDTFeatureSelector.threshold': 0.8071105364595893, 'num_leaves': 8, 'max_depth': 9}
21.16031987663184


# モデル制作

In [ ]:
target = df_num["pm25_mid"]
cols=selector.get_selected_cols()

df_train=df_num
df_test=test_num

In [ ]:
for i in cols:
  print(i)
  if i not in df_test.columns:
    df_test[i]=0

co_mid
month
no2_mid
no2_min
o3_mid
co_min
so2_mid
o3_max
so2_min
co_var
o3_var
pressure_min
temperature_max
ws_min
temperature_mid
humidity_mid
id
co_max
so2_var
pressure_max
so2_max
Ürümqi
humidity_min
o3_min
so2_cnt
no2_var
pressure_cnt
Lampang
temperature_min
no2_cnt
dew_min
ws_mid
dew_max
Vereeniging
o3_cnt
pressure_var
no2_max
Chandigarh
Bhopal
ws_cnt
dew_mid
Delhi
ws_var
dew_cnt
Lhasa
Gandhinagar
Novosibirsk
temperature_cnt
humidity_cnt
ws_max
temperature_var
co_cnt
Bandar Abbas
Shillong
lat
Qingdao
Zanjān
San Jose
Adapazarı
humidity_max
Samsun
Lucknow
Mumbai
Antakya
Worcester
Jerusalem
Jinan
Patna
Hyderabad
Changsha
Calama
Thiruvananthapuram
Toluca
Jackson
Nanchang
Phoenix
pressure_mid
Guadalajara
Visakhapatnam
Oaxaca
Denver
Muzaffarnagar
Talca
Vitória
Warsaw
Beijing
Raleigh
Vancouver
Suncheon
Hạ Long
Qinhuangdao
East London
dew_var
Nakhon Pathom
San Luis Potosí
lon
Qiqihar
Middelburg
Budapest
Miskolc
Akita
Xuchang
Hāpur
Istanbul
Kolkata
Port Elizabeth
Hanoi
Eskişehir
Milwaukee

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


In [ ]:
# parameters

learning_rate = 0.15
SEED = 42
lgb_params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": learning_rate,
    "num_leaves": 8,
    "min_data_in_leaf": 40,
    #"colsample_bytree": 1.0,
    #"feature_fraction": 1.0,
    #"bagging_freq": 0,
    #"bagging_fraction": 1.0,
    "verbosity": -1,
    "seed": SEED,
}

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np

SEED_SKF = SEED
np.random.seed(SEED)
 
FOLD = 5
skf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=SEED_SKF)
kf = KFold(n_splits=FOLD, shuffle=True, random_state=SEED)

oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
df_feature_importance = pd.DataFrame()

features = cols 

N_CLASSES=1

In [ ]:
for fold_, (indexes_trn, indexes_val) in enumerate(kf.split(df_train.values, target.values)):
    print(f"------------------------------ fold {fold_} ------------------------------")
    
    df_trn = df_train.loc[indexes_trn].reset_index(drop=True)
    df_val = df_train.loc[indexes_val].reset_index(drop=True)
    target_trn = target.loc[indexes_trn].reset_index(drop=True)
    target_val = target.loc[indexes_val].reset_index(drop=True)
    
    
    lgb_train = lgb.Dataset(
        df_trn.loc[:, features],
        label=target_trn,
        feature_name=features,
    )
    lgb_valid = lgb.Dataset(
        df_val.loc[:, features],
        label=target_val,
        feature_name=features,
    )
    
    lgb_params["learning_rate"] = learning_rate + np.random.random() * 0.001  # おまじない
    num_round = 999999999
    model = lgb.train(
        lgb_params,
        lgb_train, 
        num_round, 
        valid_sets=[lgb_train, lgb_valid], 
        verbose_eval=300,
        early_stopping_rounds=300 if num_round >= 1e8 else None,
        fobj=None,
        #feval=lgb_metric,
    )
    
    # cv
    prediction_round = model.best_iteration+150 if num_round >= 1e8 else num_round  # おまじない
    oof[indexes_val] = model.predict(df_val[features], num_iteration=prediction_round) 
    
    # feature importance
    df_fold_importance = pd.DataFrame()
    df_fold_importance["feature"] = features
    df_fold_importance["importance"] = model.feature_importance()
    df_fold_importance["fold"] = fold_

    # prediction for test data
    predictions += model.predict(df_test[features], num_iteration=prediction_round) / FOLD
    print()

------------------------------ fold 0 ------------------------------
Training until validation scores don't improve for 300 rounds.
[300]	training's rmse: 19.9285	valid_1's rmse: 20.5167
[600]	training's rmse: 19.0619	valid_1's rmse: 20.0418
[900]	training's rmse: 18.4604	valid_1's rmse: 19.8358
[1200]	training's rmse: 17.9482	valid_1's rmse: 19.7015
[1500]	training's rmse: 17.5135	valid_1's rmse: 19.6117
[1800]	training's rmse: 17.1142	valid_1's rmse: 19.5444
[2100]	training's rmse: 16.7479	valid_1's rmse: 19.4871
[2400]	training's rmse: 16.411	valid_1's rmse: 19.4431
[2700]	training's rmse: 16.0814	valid_1's rmse: 19.4271
[3000]	training's rmse: 15.7774	valid_1's rmse: 19.407
[3300]	training's rmse: 15.4885	valid_1's rmse: 19.3986
[3600]	training's rmse: 15.2232	valid_1's rmse: 19.3886
[3900]	training's rmse: 14.959	valid_1's rmse: 19.375
[4200]	training's rmse: 14.7084	valid_1's rmse: 19.3702
[4500]	training's rmse: 14.4662	valid_1's rmse: 19.3613
[4800]	training's rmse: 14.2368	val

In [ ]:
pd.set_option('display.max_rows', 500)
df_fold_importance 

,feature,importance,fold
0,co_mid,1111,4
1,month,582,4
2,no2_mid,980,4
3,no2_min,1011,4
4,o3_mid,725,4
...,...,...,...
516,Sanandaj,5,4
517,Marseille,21,4
518,Zhuzhou,16,4
519,Zhengzhou,16,4


In [ ]:
from sklearn.metrics import mean_squared_error

df_test["prediction"] = predictions.tolist()
score=np.sqrt(mean_squared_error(target, oof))
print("CV score")
print(f"  rmse: {score:8.5f}")

CV score
  rmse: 19.30635


In [ ]:
sample=sample.set_axis(['id','pm25_mid'],axis=1)

In [ ]:
df_submission = sample.copy()
df_submission["pm25_mid"] = df_submission["id"].map(dict(df_test[["id", "prediction"]].values))
assert not df_submission["pm25_mid"].isna().any()
display(df_submission.head(10))

# make submission file
df_submission.to_csv("v2.csv",header=None,index=None)

,id,pm25_mid
0,195942,19.991359
1,195943,39.794109
2,195944,26.817890
3,195945,43.748692
4,195946,128.211875
5,195947,24.367190
6,195948,108.758680
7,195949,65.788008
8,195950,54.298009
9,195951,122.204950
